In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [2]:
# 크롤링 문서 요청해서 응답객체 반환받기
url="http://jolse.com/category/toners-mists/1019/"
html = urlopen(url)
htmls = html.read()

# print(htmls)
bs_obj = BeautifulSoup(htmls,"html.parser")

In [6]:
## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
# ul class:prdList grid4
ul=bs_obj.find("ul",{"class":"prdList grid4"})

## 품목 1개를 담고 있는 div 태그 추출
## div class:box
prd_boxes = ul.findAll("div", {"class":"box"})

In [4]:
# box 안에 들어 있는 1개의 상품에서 정보를 추출해서 dict 형태로 반환 해 주는 함수 작성

def get_product_info(box) :

    # 제품명 추출 코드
    p_tag = box.find("p",{"class":"name"})
    span = p_tag.find("span")
    # 가격 추출 코드
    price_ul = box.find("ul")
    price_span = price_ul.findAll("span")
    return{"prd_name":span.text, "price":price_span[1].text, "sale_price":price_span[-1].text}

In [7]:
for box in prd_boxes :
    product_info = get_product_info(box)
    print(product_info)

{'prd_name': 'Isntree Hyaluronic Acid Toner 200ml (Renewal)', 'price': 'USD 19.00', 'sale_price': 'USD 13.99'}
{'prd_name': 'SON&PARK BEAUTY WATER 340ml', 'price': 'USD 29.40', 'sale_price': 'USD 24.99'}
{'prd_name': 'SOME BY MI AHA BHA PHA 30 Days Miracle Toner 150ml', 'price': 'USD 24.00', 'sale_price': 'USD 13.99'}
{'prd_name': 'SOME BY MI Galactomyces Pure Vitamin C Glow Toner 200ml', 'price': 'USD 34.00', 'sale_price': 'USD 17.99'}
{'prd_name': 'PURITO Centella Green Level Calming Toner 200ml', 'price': 'USD 25.20', 'sale_price': 'USD 21.42'}
{'prd_name': 'SON&PARK Beauty Water 60ml', 'price': 'USD 10.50', 'sale_price': 'USD 8.92'}
{'prd_name': 'Pyunkang Yul Essence Toner 100ml', 'price': 'USD 14.50', 'sale_price': 'USD 12.32'}
{'prd_name': 'secretKey Hyaluron Aqua Soft Toner 500ml', 'price': 'USD 22.00', 'sale_price': 'USD 18.70'}
{'prd_name': 'ROUND LAB 1025 Dokdo Toner 200ml', 'price': 'USD 17.00', 'sale_price': 'USD 14.45'}
{'prd_name': 'SOME BY MI Snail Truecica Miracle Repai

In [8]:
# 전달된 url 페이지에 접근해서 해당 페이지의 전체 상품 데이터를 추출한 후 각 상품마다 get_product_info() 함수를 호출해서
# 각 상품에 대한 추출 정보를 받아 옴 - 들어 온 각 상품들의 정보를 리스트에 저장한 후 해당 리스트를 반환
def get_page_products(url) :
    
    # 전달받은 url 열기
    url=url
    html = urlopen(url)
    htmls = html.read()

    # url을 bs4 obj로 반환
    bs_obj = BeautifulSoup(htmls,"html.parser")
    
    ## 한 페이지에 모든 상품이 들어있는 ul 태그 추출
    # ul class:prdList grid4
    ul=bs_obj.find("ul",{"class":"prdList grid4"})

    ## 품목 1개를 담고 있는 div 태그 추출
    ## div class:box
    prd_boxes = ul.findAll("div", {"class":"box"}) # 1개의 페이지의 전체 상품
    
    prod_info_list = [get_product_info(box) for box in prd_boxes] # 현재 함수를 통해 접속한 문서(url인수)의 각 화장품 정보가
    # 저장되어 있는 리스트임
    
    return prod_info_list

In [12]:
# 여러 페이지의 화장품 정보를 추출하여 df에 저장 후 csv에 저장하는 코드
url = "http://jolse.com/category/toners-mists/1019/?page=" # 페이지 번호를 제외한 공통 url 문자열

df_fin = pd.DataFrame() # column이 구성되어 있지 않기 때문에 append가 안 됨 따라서 concact 사용
for i in range(1,6) :
    # url 완성 : page 번호를 추가 - i 변수 값을 활용
    urlfin = url+ str(i)
    page_product = get_page_products(urlfin)
    df = pd.DataFrame(page_product) # 현재 url로 접속한 페이지에 대해 수집된 제품정보를 df로 생성(한 개 페이지 제품)
    df_fin = pd.concat([df_fin,df], axis=0, ignore_index=True)

df_fin

,prd_name,price,sale_price
0,Isntree Hyaluronic Acid Toner 200ml (Renewal),USD 19.00,USD 13.99
1,SON&PARK BEAUTY WATER 340ml,USD 29.40,USD 24.99
2,SOME BY MI AHA BHA PHA 30 Days Miracle Toner 1...,USD 24.00,USD 13.99
3,SOME BY MI Galactomyces Pure Vitamin C Glow To...,USD 34.00,USD 17.99
4,PURITO Centella Green Level Calming Toner 200ml,USD 25.20,USD 21.42
...,...,...,...
235,innisfree Truecare PHA Toner Pad 70ea,USD 23.32,USD 19.82
236,innisfree Truecare Panthenol 10 Moisture Skin ...,USD 22.56,USD 19.18
237,VELY VELY Pink Peeling Pad 60ea,USD 27.06,USD 23.00
238,VELY VELY Madecassoside Repair Toner 150ml,USD 34.12,USD 29.00


In [13]:
df_fin.to_csv('./crawl_data/prod0805All.csv')

### 연습문제

- 1. --- 품목 상세보기 링크도 추가해서 크롤링
    - 제품명 바깥에 걸려 있는 a 태그에서 href 속성의 상세보기 링크
- 2. --- 사이트내에서 전체페이지를 확인할 수 있는 태그가 있는지 검사하고 해당 태그에서 총 페이지를 추출한 후
-  전체 페이지를 크롤링 할 수 있는 프로그램으로 변경
    - 맨 끝 페이지로 넘어가는 버튼 내 a 태그의 href 속성에 있는 링크 추출하는 코드 짜보기
        - href 가져오고 어떤 기호를 사용해서 split -> 마지막 페이지 숫자만 추출
